In [4]:
cd 'drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car'

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car'
/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [9]:
!pip install eli5

In [0]:
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
## Wczytywanie danych

In [0]:
df = pd.read_hdf('data/car.h5')

In [15]:
df.shape

(106494, 155)

Dummy Model

In [16]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [20]:
feats = ['car_id']
X = df[feats].values
Y = df['price_value'].values

model = DummyRegressor()
model.fit(X,Y)
y_pred = model.predict(X)


mean_absolute_error(Y, y_pred)

39465.934630440985

In [21]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [22]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [23]:
df=df[df['price_currency'] != 'EUR']
df.shape

(106290, 155)

## Features

In [27]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list):continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat +SUFFIX_CAT] = factorized_values

In [33]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [38]:
X = df[cat_feats].values
Y = df['price_value'].values

model = DecisionTreeRegressor(max_depth = 6)
scores = cross_val_score(model, X, Y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-18386.025412044215

In [40]:
m = DecisionTreeRegressor(max_depth = 6)
m.fit(X,Y)

imp= PermutationImportance(m, random_state = 0).fit(X,Y)
eli5.show_weights(m, feature_names=cat_feats)

Weight,Feature
0.3302,param_faktura-vat__cat
0.2817,param_napęd__cat
0.0950,param_rok-produkcji__cat
0.0616,feature_kamera-cofania__cat
0.0521,param_stan__cat
0.0496,param_moc__cat
0.0300,param_skrzynia-biegów__cat
0.0177,param_pojemność-skokowa__cat
0.0176,feature_łopatki-zmiany-biegów__cat
0.0167,feature_bluetooth__cat
